### Model Optimizasyonu ve Overfitting Yönetimi
Selin Hoca'nın slaytındaki 'Overfitting' riskini yönetmek ve modelin genelleme yeteneğini artırmak için Optuna ile hiperparametre optimizasyonu yaptım.

Validation setini kullanarak `learning_rate`, `num_leaves` ve regülasyon (`lambda`) parametrelerini bilimsel olarak ayarladım.

In [1]:
import pandas as pd
import numpy as np
import optuna

In [2]:
import os
import joblib

In [3]:
train_df = pd.read_csv('../data/ranking/train.csv')
val_df = pd.read_csv('../data/ranking/val.csv')

In [4]:
q_train = train_df.groupby('Event_Block', sort=False).size().to_list()
q_val = val_df.groupby('Event_Block', sort=False).size().to_list()

In [5]:
drop_cols = ['Class', 'Time', 'Event_Block']

In [6]:
X_train = train_df.drop(drop_cols, axis=1)
y_train = train_df['Class']

In [7]:
X_val = val_df.drop(drop_cols, axis=1)
y_val = val_df['Class']

In [8]:
print(f"Train Gruplari: {len(q_train)} adet")
print(f"Val Gruplari: {len(q_val)} adet")

Train Gruplari: 33 adet
Val Gruplari: 7 adet


In [9]:
import lightgbm as lgb
from sklearn.metrics import ndcg_score

In [16]:
def objective(trial):
    params = {
        'objective': 'lambdarank', 
        'metric': 'ndcg',          
        'eval_at': 5,              
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
    }

    model = lgb.LGBMRanker(**params)

    model.fit(
        X_train, y_train,
        group=q_train,
        eval_set=[(X_val, y_val)],
        eval_group=[q_val],
        callbacks=[lgb.early_stopping(stopping_rounds=50)] 
    )

    score = model.best_score_['valid_0']['ndcg@5']
    return score

In [31]:
print("Optimizasyon başlıyor... (Bu işlem birkaç dakika sürebilir)")
sampler = optuna.samplers.TPESampler(seed=42) # hep ayni sirayla arama yapilsin diye sampler a seed degiskeni verdim
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=20)

[I 2025-12-03 20:53:42,555] A new study created in memory with name: no-name-eff94eb2-ae28-49b8-92da-9d79fe87b8cb


Optimizasyon başlıyor... (Bu işlem birkaç dakika sürebilir)
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-03 20:53:43,242] Trial 0 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.08116262258099888, 'num_leaves': 144, 'lambda_l1': 7.319939418114051, 'lambda_l2': 5.986584841970366, 'bagging_fraction': 0.5780093202212182, 'feature_fraction': 0.5779972601681014}. Best is trial 0 with value: 0.9603895103778503.


Did not meet early stopping. Best iteration is:
[63]	valid_0's ndcg@5: 0.96039
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-03 20:53:43,656] Trial 1 finished with value: 0.9174201989279717 and parameters: {'learning_rate': 0.021035886311957897, 'num_leaves': 133, 'lambda_l1': 6.011150117432088, 'lambda_l2': 7.080725777960454, 'bagging_fraction': 0.5102922471479012, 'feature_fraction': 0.9849549260809971}. Best is trial 0 with value: 0.9603895103778503.


...

Early stopping, best iteration is:
[38]	valid_0's ndcg@5: 0.981256
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-03 20:53:55,158] Trial 18 finished with value: 1.0 and parameters: {'learning_rate': 0.17164924085214237, 'num_leaves': 41, 'lambda_l1': 1.3787401756220243, 'lambda_l2': 5.0902714455019344, 'bagging_fraction': 0.7851232997829733, 'feature_fraction': 0.9003707193018444}. Best is trial 2 with value: 1.0.


Early stopping, best iteration is:
[39]	valid_0's ndcg@5: 1
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-03 20:53:56,142] Trial 19 finished with value: 0.9812564174982369 and parameters: {'learning_rate': 0.18170292925343698, 'num_leaves': 56, 'lambda_l1': 0.004795398173113807, 'lambda_l2': 0.28687726743779063, 'bagging_fraction': 0.8404619928673609, 'feature_fraction': 0.606315181465154}. Best is trial 2 with value: 1.0.


Early stopping, best iteration is:
[18]	valid_0's ndcg@5: 0.981256


In [32]:
print("\n------------------------------------------------")
print("EN İYİ PARAMETRELER:", study.best_params)
print("EN İYİ SKOR (NDCG@5):", study.best_value)
print("------------------------------------------------")


------------------------------------------------
EN İYİ PARAMETRELER: {'learning_rate': 0.16816410175208013, 'num_leaves': 47, 'lambda_l1': 1.8182496720710062, 'lambda_l2': 1.8340450985343382, 'bagging_fraction': 0.6521211214797689, 'feature_fraction': 0.762378215816119}
EN İYİ SKOR (NDCG@5): 1.0
------------------------------------------------


In [33]:
best_params = study.best_params

In [34]:
best_params['objective'] = 'lambdarank'
best_params['metric'] = 'ndcg'
best_params['eval_at'] = 5
best_params['random_state'] = 42

In [35]:
final_model = lgb.LGBMRanker(**best_params)

In [36]:
final_model.fit(
    X_train, y_train,
    group=q_train,
    eval_set=[(X_val, y_val)],
    eval_group=[q_val],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Early stopping, best iteration is:
[45]	valid_0's ndcg@5: 1


,boosting_type,'gbdt'
,num_leaves,47
,max_depth,-1
,learning_rate,0.16816410175208013
,n_estimators,100
,subsample_for_bin,200000
,objective,'lambdarank'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [37]:
os.makedirs('../models', exist_ok=True)
joblib.dump(final_model, '../models/lgbm_ranker.pkl')

['../models/lgbm_ranker.pkl']